In [3]:
# import INSTANCE_ID and LAYER_NAME from config file
from sentinel_hub_config import *

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import glob
import cv2

import gzip
import numpy

import os.path
from os import path

from sentinelhub import WmsRequest, BBox, CRS, MimeType, CustomUrlParam, get_area_dates
from s2cloudless import S2PixelCloudDetector, CloudMaskRequest
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [6]:
!pwd

/home/brendan/sdthon/code/1_raw


In [7]:
coords = pd.read_csv("../../data/1_raw/coordinates.csv")
tiles = pd.read_csv("../../data/1_raw/tiles.csv")
dates = pd.read_csv("../../data/1_raw/dates.csv")

In [8]:
coords.head()
tiles.head()
#dates.head()

,tile,N_dates,tile_x,tile_y
0,1536_1024,71,1536,1024
1,1536_1536,71,1536,1536
2,2048_1024,71,2048,1024
3,2048_1536,71,2048,1536
4,4608_4608,71,4608,4608


In [9]:
m = pd.merge(tiles, coords, how='left', on=['tile_x', 'tile_y'])

In [10]:
m.shape
m.head()
m = m[['tile_x', 'tile_y', 'x1', 'y1', 'x3', 'y3']]

In [11]:
m.shape[0]

65

In [12]:
os.chdir("../../data/phase-02")

In [13]:
m.shape

(65, 6)

In [14]:
!pwd

/home/brendan/sdthon/data/phase-02


In [15]:
def get_img(date_str, x, y, bbox_coords_wgs84):

    bounding_box = BBox(bbox_coords_wgs84, crs=CRS.WGS84)

    wms_true_color_request = WmsRequest(layer=LAYER_NAME,
                                    bbox=bounding_box, 
                                    time=(date_str), 
                                    width=512, height=512,
                                    image_format=MimeType.PNG,
                                    instance_id=INSTANCE_ID)

    date = wms_true_color_request.get_dates()
    print(date)
    wms_true_color_imgs = wms_true_color_request.get_data()

    plt.imsave(f'{x}-{y}-img-{date_str}.png', wms_true_color_imgs[0])

In [16]:
dates.head()

,date,N_tiles
0,2016-12-22,65
1,2017-01-01,65
2,2017-01-11,65
3,2017-02-10,65
4,2017-02-20,65


In [17]:
dates_array = dates['date']

In [18]:
x = 7680
y = 10240
bbox_coords_wgs84 = [148.69880265667112,-20.81291008367845, 148.74841411882176, -20.858591175163056]
date_str = '2016-12-22'
    
get_img(date_str, x, y, bbox_coords_wgs84)

[datetime.datetime(2016, 12, 22, 0, 20, 59)]


In [19]:
cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2)

In [20]:
def get_bands(date_str, x, y, bbox_coords_wgs84):

    bands_script = 'return [B01,B02,B04,B05,B08,B8A,B09,B10,B11,B12]'

    bounding_box = BBox(bbox_coords_wgs84, crs=CRS.WGS84)

    wms_bands_request = WmsRequest(layer=LAYER_NAME,
                               custom_url_params={
                                   CustomUrlParam.EVALSCRIPT: bands_script,
                                   CustomUrlParam.ATMFILTER: 'NONE'
                               },
                               bbox=bounding_box, 
                               time=(date_str), 
                               width=512, height=512,
                               image_format=MimeType.TIFF_d32f,
                               instance_id=INSTANCE_ID)
    
    wms_bands = wms_bands_request.get_data()
    
    #print(len(wms_bands))
    #print(wms_bands[0].shape)
    
    return(wms_bands)

In [21]:
x = 7680
y = 10240
bbox_coords_wgs84 = [148.69880265667112,-20.81291008367845, 148.74841411882176, -20.858591175163056]
date_str = '2016-12-22'

def get_cloud_all_dates(x, y, bbox_coords_wgs84, dates_array):

    for date_str in dates_array:
        if not path.exists(f'{x}-{y}-mask-{date_str}.png') and not path.exists(f'{x}-{y}-prob-{date_str}.png'):
            print(f'x{x}-y{y}-{date_str}')
            wms_bands = get_bands(date_str, x, y, bbox_coords_wgs84)

            cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(wms_bands))
            cloud_masks = cloud_detector.get_cloud_masks(np.array(wms_bands))
            cloud_probs = cloud_probs.round(decimals=2, out=None)
            plt.imsave(f'{x}-{y}-prob-{date_str}.png', cloud_probs[0])
            plt.imsave(f'{x}-{y}-mask-{date_str}.png', cloud_masks[0])
            f = gzip.GzipFile(f"{x}-{y}-prob-{date_str}.npy.gz", "w")
            numpy.save(file=f, arr=cloud_probs[0])
            f.close()

In [22]:
def get_cloud_cover(x, y, bbox_coords_wgs84, dates_array):

    colnames = ['cc_p20', 'cc_p40', 'cc_p60', 'cc_p80', 'cc_default', 'date', 'x', 'y']
    for date_str in dates_array:
        if not path.exists(f'{x}-{y}-cloud-{date_str}.csv'):
            print(f'x{x}-y{y}-{date_str}')
            wms_bands = get_bands(date_str, x, y, bbox_coords_wgs84)

            cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(wms_bands))
            cloud_masks = cloud_detector.get_cloud_masks(np.array(wms_bands))

            cc_p20 = np.sum(np.array([np.where(cloud_probs[0] > 0.2, 1, 0)])[0])                    
            cc_p40 = np.sum(np.array([np.where(cloud_probs[0] > 0.4, 1, 0)])[0])
            cc_p60 = np.sum(np.array([np.where(cloud_probs[0] > 0.6, 1, 0)])[0])        
            cc_p80 = np.sum(np.array([np.where(cloud_probs[0] > 0.8, 1, 0)])[0])

            cc_default = np.sum(cloud_masks[0])
            r = pd.DataFrame([[cc_p20, cc_p40, cc_p60, cc_p80, cc_default, date_str, x, y]], columns= colnames)
            r.to_csv(f'{x}-{y}-cloud-{date_str}.csv')
            
            cloud_probs = cloud_probs.round(decimals=2, out=None)
            plt.imsave(f'{x}-{y}-prob-{date_str}.png', cloud_probs[0])
            plt.imsave(f'{x}-{y}-mask-{date_str}.png', cloud_masks[0])
            f = gzip.GzipFile(f"{x}-{y}-prob-{date_str}.npy.gz", "w")
            numpy.save(file=f, arr=cloud_probs[0])
            f.close()


In [ ]:
for i in range(2, m.shape[0]):
    bbox_coords_wgs84 = m.loc[[i,]][['x1','y1','x3','y3']].values.tolist()[0]
    x = m.loc[[i,]][['tile_x','tile_y']].values.tolist()[0][0]
    y = m.loc[[i,]][['tile_x','tile_y']].values.tolist()[0][1]
    print(f"i:{i}, x:{x}, y:{y}, {bbox_coords_wgs84}")
    if i != 59:
        get_cloud_cover(x, y, bbox_coords_wgs84, dates_array)
        #get_cloud_all_dates(x, y, bbox_coords_wgs84, dates_array)
    

i:2, x:2048, y:1024, [148.1515676203218, -19.984663182667305, 148.20075266660697, -20.030503449080147]
x2048-y1024-2016-12-22
x2048-y1024-2017-01-01
x2048-y1024-2017-01-11
x2048-y1024-2017-02-10
x2048-y1024-2017-02-20
x2048-y1024-2017-03-02
x2048-y1024-2017-03-12
x2048-y1024-2017-04-01
x2048-y1024-2017-04-11
x2048-y1024-2017-05-01
x2048-y1024-2017-05-11
x2048-y1024-2017-05-21
x2048-y1024-2017-05-31
x2048-y1024-2017-06-20
x2048-y1024-2017-07-10
x2048-y1024-2017-07-20
x2048-y1024-2017-07-30
x2048-y1024-2017-08-09
x2048-y1024-2017-08-19
x2048-y1024-2017-09-08
x2048-y1024-2017-09-18
x2048-y1024-2017-09-28
x2048-y1024-2017-10-08
x2048-y1024-2017-10-28
x2048-y1024-2017-11-07
x2048-y1024-2017-11-17
x2048-y1024-2017-11-27
x2048-y1024-2017-12-07
x2048-y1024-2017-12-17
x2048-y1024-2017-12-27
x2048-y1024-2018-01-06
x2048-y1024-2018-01-26
x2048-y1024-2018-02-15
x2048-y1024-2018-03-17
x2048-y1024-2018-04-16
x2048-y1024-2018-04-26
x2048-y1024-2018-05-06
x2048-y1024-2018-05-26
x2048-y1024-2018-06-05


In [ ]:
date_str='2017-05-31'
x=1536 
y=1536
bbox_coords_wgs84=[148.10297228244568, -20.031237148182274, 148.1521576054924, -20.077091525735625]

In [ ]:
"""
wms_bands = get_bands(date_str, x, y, bbox_coords_wgs84)

cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(wms_bands))
cloud_masks = cloud_detector.get_cloud_masks(np.array(wms_bands))
cloud_probs = cloud_probs.round(decimals=2, out=None)
plt.imsave(f'{x}-{y}-prob-{date_str}.png', cloud_probs[0])
plt.imsave(f'{x}-{y}-mask-{date_str}.png', cloud_masks[0])
f = gzip.GzipFile(f"{x}-{y}-prob-{date_str}.npy.gz", "w")
numpy.save(file=f, arr=cloud_probs[0])
f.close()
"""


In [ ]:
"""
cloud_probs.shape
cloud_probs[0]
cloud_masks.shape
cloud_masks[0]
cloud_probs = cloud_probs.round(decimals=2, out=None)
plt.imsave(f'{x}-{y}-prob-{date_str}.png', cloud_probs[0])
plt.imsave(f'{x}-{y}-mask-{date_str}.png', cloud_masks[0])
"""

In [ ]:
cloud_probs[0]

In [ ]:
"""
import gzip
import numpy
f = gzip.GzipFile(f"{x}-{y}-prob-{date_str}.npy.gz", "w")
numpy.save(file=f, arr=cloud_probs[0])
f.close()
"""


In [ ]:
f = gzip.GzipFile(f"{x}-{y}-prob-{date_str}.npy.gz", "r")
a = np.load(f)

In [ ]:
pd.DataFrame(a)

In [ ]:
#f.close()


In [ ]:
"""
import h5py, numpy as np

#arr = np.random.randint(0, 10, (1000, 1000))

f = h5py.File('file.h5', 'w', libver='latest')  # use 'latest' for performance

dset = f.create_dataset('array', shape=(512, 512), data=cloud_probs[0], chunks=(100, 100),
                        compression='gzip', compression_opts=9)

f.close()
#https://stackoverflow.com/questions/49740190/saving-in-a-file-an-array-or-dataframe-together-with-other-information
"""


In [ ]:

"""
pd.DataFrame(a).to_hdf('file2.h5', 'table', mode='w',append=True, complevel=9, complib='zlib')

#https://dziganto.github.io/out-of-core%20computation/HDF5-Or-How-I-Learned-To-Love-Data-Compression-And-Partial-Input-Output/
compressors = ['blosc', 'bzip2', 'zlib']
for compressor in compressors:
    X_train.to_hdf('filepath_' + str(compressor) + '.h5', 
                   'table', mode='w', append=True, complevel=9, complib=compressor)
    X_test.to_hdf('filepath_' + str(compressor) + '.h5', 
                   'table', mode='w', append=True, complevel=9, complib=compressor)
"""


In [ ]:
cloud_masks[0].shape

In [ ]:
#np.savetxt('x.txt',cloud_probs[0])

In [ ]:
#cv2.imwrite(f'{x}-{y}-prob-{date_str}.png', cloud_probs[0])


In [ ]:
#plt.imshow(cloud_probs[0])


In [ ]:
#cp = cv2.imread(f'{x}-{y}-prob-{date_str}.png')
#cp

In [ ]:
#def rgb2gray(rgb):
#    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

#gray = rgb2gray(cp)    
#gray/255

In [ ]:
#plt.imshow(gray)
#cp

In [ ]:
"""
from PIL import Image
from numpy import asarray
image = Image.open(f'{x}-{y}-prob-{date_str}.png')
#plt.imshow(image)
gs_image = image.convert(mode='L')
plt.imshow(gs_image)
asarray(gs_image)
"""


In [ ]:
cm = np.array([np.where(cloud_probs[0] > 0.4, 1, 0)])

In [ ]:
np.sum(cm[0])

In [ ]:
#cloud_masks = cloud_detector.get_cloud_masks(np.array(wms_bands))

In [ ]:
#cloud_masks.shape

In [ ]:
#np.sum(cloud_masks[0])

In [ ]:
#cm.shape

In [ ]:
image_idx = 0
#plot_probability_map(wms_true_color_imgs[image_idx], cloud_probs[image_idx])

In [ ]:
image_idx = 0
#overlay_cloud_mask(wms_true_color_imgs[image_idx], cm[image_idx])

In [ ]:
image_idx = 0
#overlay_cloud_mask(wms_true_color_imgs[image_idx], cloud_masks[image_idx])

In [ ]:
len(wms_bands)

In [ ]:
wms_bands[0]